# 05: LoRa Deep Dive

This notebook explores LoRa's Chirp Spread Spectrum (CSS) modulation in detail.

## Learning Objectives
- Understand chirp spread spectrum modulation
- Analyze spreading factor effects
- Visualize LoRa signals in time and frequency
- Explore the sensitivity vs data rate tradeoff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from r4w_python import plot_waterfall, compute_waterfall

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Chirp Spread Spectrum Basics

LoRa uses **chirps** - signals whose frequency changes linearly over time:

- **Upchirp**: Frequency increases from $-BW/2$ to $+BW/2$
- **Downchirp**: Frequency decreases from $+BW/2$ to $-BW/2$

Data is encoded by **shifting** the starting frequency of the chirp.

In [ ]:
def generate_chirp(sf, bw, sample_rate, symbol=0, direction='up'):
    """Generate a LoRa chirp.
    
    Args:
        sf: Spreading factor (7-12)
        bw: Bandwidth in Hz
        sample_rate: Sample rate in Hz
        symbol: Symbol value (0 to 2^SF - 1)
        direction: 'up' or 'down'
    """
    n_samples = int(sample_rate * (2**sf) / bw)
    t = np.arange(n_samples) / sample_rate
    
    # Chirp rate
    k = bw / (2**sf / bw)  # Hz per second
    
    # Symbol offset
    f_offset = symbol * bw / (2**sf)
    
    if direction == 'up':
        freq = -bw/2 + f_offset + k * t
        freq = np.mod(freq + bw/2, bw) - bw/2  # Wrap around
    else:
        freq = bw/2 - f_offset - k * t
        freq = np.mod(freq + bw/2, bw) - bw/2
    
    phase = 2 * np.pi * np.cumsum(freq) / sample_rate
    return np.exp(1j * phase)

# Parameters
sf = 7
bw = 125000  # 125 kHz
sample_rate = bw * 2  # Nyquist

In [ ]:
# Generate upchirp and downchirp
upchirp = generate_chirp(sf, bw, sample_rate, symbol=0, direction='up')
downchirp = generate_chirp(sf, bw, sample_rate, symbol=0, direction='down')

# Time domain
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

t_ms = np.arange(len(upchirp)) / sample_rate * 1000

axes[0, 0].plot(t_ms, upchirp.real)
axes[0, 0].set_title('Upchirp - Time Domain')
axes[0, 0].set_xlabel('Time (ms)')

axes[0, 1].plot(t_ms, downchirp.real)
axes[0, 1].set_title('Downchirp - Time Domain')
axes[0, 1].set_xlabel('Time (ms)')

# Spectrograms
for ax, sig, title in [(axes[1, 0], upchirp, 'Upchirp Spectrogram'),
                        (axes[1, 1], downchirp, 'Downchirp Spectrogram')]:
    wf = compute_waterfall(sig, fft_size=64, hop_size=16)
    im = ax.imshow(wf, aspect='auto', origin='lower', cmap='viridis',
                   extent=[-bw/2000, bw/2000, 0, len(sig)/sample_rate*1000])
    ax.set_xlabel('Frequency (kHz)')
    ax.set_ylabel('Time (ms)')
    ax.set_title(title)

plt.tight_layout()
plt.show()

## Symbol Encoding

Each symbol shifts the chirp starting frequency:
- Symbol 0: Start at $-BW/2$
- Symbol N: Start at $-BW/2 + N \cdot BW/2^{SF}$

For SF=7: 128 possible symbols (7 bits per symbol)

In [ ]:
# Generate different symbols
symbols_to_show = [0, 32, 64, 96]
chirps = [generate_chirp(sf, bw, sample_rate, s) for s in symbols_to_show]
combined = np.concatenate(chirps)

# Waterfall
wf = compute_waterfall(combined, fft_size=64, hop_size=16)

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.imshow(wf, aspect='auto', origin='lower', cmap='viridis',
               extent=[-bw/2000, bw/2000, 0, len(combined)/sample_rate*1000])
ax.set_xlabel('Frequency (kHz)')
ax.set_ylabel('Time (ms)')
ax.set_title(f'LoRa SF{sf} Symbols: {symbols_to_show}')
plt.colorbar(im, label='Power (dB)')

# Mark symbol boundaries
symbol_duration_ms = len(upchirp) / sample_rate * 1000
for i, s in enumerate(symbols_to_show):
    ax.axhline((i+1) * symbol_duration_ms, color='r', linestyle='--', alpha=0.5)
    ax.text(bw/2000 + 5, (i + 0.5) * symbol_duration_ms, f'Symbol {s}', color='white')

plt.show()

## Demodulation: Dechirping

To demodulate, multiply by conjugate downchirp:

$$y[n] = x[n] \cdot \text{downchirp}^*[n]$$

This converts the chirp to a tone at frequency proportional to the symbol value.

In [ ]:
def demodulate_symbol(chirp, downchirp):
    """Demodulate a LoRa symbol using FFT."""
    # Dechirp
    dechirped = chirp * np.conj(downchirp)
    
    # FFT to find peak
    spectrum = np.fft.fft(dechirped)
    peak_bin = np.argmax(np.abs(spectrum))
    
    # Wrap to first half
    if peak_bin > len(spectrum) // 2:
        peak_bin = len(spectrum) - peak_bin
    
    return peak_bin, spectrum

# Test demodulation
test_symbol = 42
tx_chirp = generate_chirp(sf, bw, sample_rate, test_symbol)
detected, spectrum = demodulate_symbol(tx_chirp, downchirp)

print(f"Transmitted symbol: {test_symbol}")
print(f"Detected symbol: {detected}")

In [ ]:
# Visualize dechirping process
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Original chirp spectrogram
wf1 = compute_waterfall(tx_chirp, fft_size=64, hop_size=16)
axes[0, 0].imshow(wf1, aspect='auto', origin='lower', cmap='viridis')
axes[0, 0].set_title(f'Received Chirp (Symbol {test_symbol})')

# Downchirp spectrogram
wf2 = compute_waterfall(downchirp, fft_size=64, hop_size=16)
axes[0, 1].imshow(wf2, aspect='auto', origin='lower', cmap='viridis')
axes[0, 1].set_title('Reference Downchirp')

# Dechirped signal (should be a tone)
dechirped = tx_chirp * np.conj(downchirp)
wf3 = compute_waterfall(dechirped, fft_size=64, hop_size=16)
axes[1, 0].imshow(wf3, aspect='auto', origin='lower', cmap='viridis')
axes[1, 0].set_title('After Dechirping (Constant Tone)')

# FFT showing peak
axes[1, 1].plot(np.abs(spectrum))
axes[1, 1].axvline(detected, color='r', linestyle='--', label=f'Peak @ bin {detected}')
axes[1, 1].set_title('FFT of Dechirped Signal')
axes[1, 1].set_xlabel('Frequency Bin')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## Spreading Factor Comparison

| SF | Bits/Symbol | Symbol Duration | Sensitivity | Data Rate |
|----|-------------|-----------------|-------------|------------|
| 7  | 7 | 1.02 ms | -123 dBm | 5.5 kbps |
| 8  | 8 | 2.05 ms | -126 dBm | 3.1 kbps |
| 9  | 9 | 4.10 ms | -129 dBm | 1.8 kbps |
| 10 | 10 | 8.19 ms | -132 dBm | 0.98 kbps |
| 11 | 11 | 16.38 ms | -134.5 dBm | 0.54 kbps |
| 12 | 12 | 32.77 ms | -137 dBm | 0.29 kbps |

In [ ]:
# Compare different spreading factors
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for ax, sf_val in zip(axes.flat, range(7, 13)):
    chirp = generate_chirp(sf_val, bw, sample_rate, symbol=0)
    wf = compute_waterfall(chirp, fft_size=64, hop_size=16)
    
    im = ax.imshow(wf, aspect='auto', origin='lower', cmap='viridis',
                   extent=[-bw/2000, bw/2000, 0, len(chirp)/sample_rate*1000])
    ax.set_xlabel('Frequency (kHz)')
    ax.set_ylabel('Time (ms)')
    ax.set_title(f'SF{sf_val}: {2**sf_val} symbols, {len(chirp)/sample_rate*1000:.1f}ms')

plt.tight_layout()
plt.show()

## Noise Immunity

LoRa can work at very low SNR due to processing gain:

$$\text{Processing Gain} = 2^{SF} = \text{Samples per symbol}$$

In [ ]:
# Test demodulation under noise
def test_noise_immunity(sf_val, snr_db, n_trials=100):
    """Test symbol error rate at given SNR."""
    errors = 0
    bw_test = 125000
    sr_test = bw_test * 2
    
    ref_down = generate_chirp(sf_val, bw_test, sr_test, 0, 'down')
    
    for _ in range(n_trials):
        symbol = np.random.randint(0, 2**sf_val)
        chirp = generate_chirp(sf_val, bw_test, sr_test, symbol)
        
        # Add noise
        signal_power = np.mean(np.abs(chirp)**2)
        noise_power = signal_power / (10**(snr_db/10))
        noise = np.sqrt(noise_power/2) * (np.random.randn(len(chirp)) + 
                                           1j * np.random.randn(len(chirp)))
        rx = chirp + noise
        
        detected, _ = demodulate_symbol(rx, ref_down)
        if detected != symbol:
            errors += 1
    
    return errors / n_trials

# Test at different SNRs
snr_range = range(-20, 5)
sf7_ser = [test_noise_immunity(7, snr, 200) for snr in snr_range]
sf10_ser = [test_noise_immunity(10, snr, 100) for snr in snr_range]

In [ ]:
plt.figure(figsize=(10, 6))
plt.semilogy(list(snr_range), sf7_ser, 'o-', label='SF7')
plt.semilogy(list(snr_range), sf10_ser, 's-', label='SF10')
plt.xlabel('SNR (dB)')
plt.ylabel('Symbol Error Rate')
plt.title('LoRa Symbol Error Rate vs SNR')
plt.legend()
plt.grid(True)
plt.ylim(1e-3, 1)
plt.show()

print("Notice: Higher SF works at lower SNR but takes longer to transmit!")

## Exercises

1. **Symbol collision**: What happens when two LoRa symbols with different values are transmitted simultaneously? (Hint: dechirp and look at FFT)

2. **Frequency offset**: How robust is dechirping to carrier frequency offset?

3. **Time on air**: Calculate the time to transmit a 50-byte packet at SF7 and SF12.

In [ ]:
# Your code here
